In [1]:
# -*- coding: utf-8 -*
import os
import re
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import keras
import warnings
from keras.utils import np_utils
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.models import Sequential, load_model
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.preprocessing.image import array_to_img, img_to_array, load_img
from keras.optimizers import RMSprop
from keras.utils import to_categorical
from keras.backend import tensorflow_backend
from keras import layers
from keras import models
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn import datasets
import scipy.linalg as LA
import cv2
import tensorflow as tf

/home/tsunei/anaconda3/envs/tf/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
config = tf.ConfigProto(gpu_options = tf.GPUOptions(allow_growth = True))
session = tf.Session(config = config)
tensorflow_backend.set_session(session)

In [3]:
def list_pictures(directory, ext='jpg|jpeg|bmp|png|ppm'):
    return [os.path.join(root, f)
            for root, _, files in os.walk(directory) for f in files
            if re.match(r'([\w]+\.(?:' + ext + '))', f.lower())]

class ZCAWhitening:
  def __init__(self, epsilon=1E-0):
    #epsilon=1E-6=81%                                                                         
    #espilon=1E-1=88.7%                                                                       
    #espilon=1E-0=88.3%                                                                      
    #espilon=10=86%                                                                            
    self.epsilon = epsilon
    self.mean = None
    self.zca = None

  def fit(self, x):
    self.mean = np.mean(x, axis=0)
    x_ = x - self.mean
    print(x_.shape)
    cov = np.dot(x_.T, x_) / x_.shape[0]
    E, D, _ = np.linalg.svd(cov)
    D = np.sqrt(D) + self.epsilon
    self.zca = np.dot(E, np.dot(np.diag(1.0 / D), E.T))
    return self

  def transform(self, x):
    x_ = x - self.mean
    return np.dot(x_, self.zca.T)

In [4]:
x = []
y = []
X_eva = []
Y_eva = []

print("人工物画像のテストデータを取得してます...")
for filepath in list_pictures('../data/CNN_data/test/j_test/'):
    warnings.filterwarnings('ignore')
    img_eva = img_to_array(load_img(filepath, grayscale=True, target_size=(128, 128))) 
    X_eva.append(img_eva)
    Y_eva.append(0) # 教師データ（正解）


print("人工物画像のテストデータを取得しました")

    
#学習データの取得（非正解画像）
print("自然物画像のテストデータを取得してます...")
for filepath in list_pictures('../data/CNN_data/test/s_test/'):
    img_eva = img_to_array(load_img(filepath, grayscale=True, target_size=(128, 128)))
    X_eva.append(img_eva)
    Y_eva.append(1) # 教師データ（正解）

print("自然物画像のテストデータを取得しました")
    
x = X_eva
y = Y_eva
X = X_eva
Y = Y_eva
# NumPy配列に変換
X_eva = np.asarray(X_eva)
Y_eva = np.asarray(Y_eva)
    
#X_eva = X_eva.reshape(X_eva.shape[0], 128, 128)

# float32型に変換
X_eva = X_eva.astype('float32')

# 正規化(0～1)
X_eva = X_eva / 255.0


人工物画像のテストデータを取得してます...
人工物画像のテストデータを取得しました
自然物画像のテストデータを取得してます...
自然物画像のテストデータを取得しました


In [5]:
#x = np.load('test_gray.npy')
#X_eva = np.load('test_gray.npy')
#Y_eva = np.load('test_label_gray.npy')
#print(Y_eva.shape)

In [6]:
print(Y_eva.shape)
print("テストデータをzca白色化しています...")
#テストデータにzca白色化
#x_zcaw_eva = X_eva.reshape(X_eva.shape[0], -1)
#zcaw_eva = ZCAWhitening().fit(x_zcaw_eva)
#x_zcaw_eva = zcaw_eva.transform(x_zcaw_eva).reshape(X_eva.shape)
datagen = ImageDataGenerator(zca_whitening=True)
g = datagen.flow(X_eva, Y_eva, X_eva.shape[0], shuffle=False)
x_zcaw_eva, Y_eva = g.next()
print(x_zcaw_eva.shape)
print(Y_eva.shape)
print("テストデータをzca白色化しました")
  
Y_eva = np_utils.to_categorical(Y_eva)
np.save('test_label_gray.npy', Y_eva)
#x_train, x_test, y_train, y_test = train_test_split(X_white, Y, test_size=0.175)
    
# 学習用データとテストデータに分割
x_test = x_zcaw_eva
y_test = Y_eva

(2000,)
テストデータをzca白色化しています...
(2000, 128, 128, 1)
(2000,)
テストデータをzca白色化しました


In [7]:
model = load_model('main_gray_temp.h5')
batch_size = 128 # バッチサイズ(データサイズ)
num_classes = 2 # 分類クラス数(今回は0～9の手書き文字なので10)
epochs = 50 # エポック数(学習の繰り返し回数)

y_pred = model.predict_classes(x_test)

sum_j = 0
sum_s = 0
for i in range(x_test.shape[0]):
    if i < 1000:
        if y_pred[i] == 0:
            sum_j = sum_j + 1
            
    else:
        if y_pred[i] == 1:
            sum_s = sum_s + 1
            
print("人工物画像",sum_j / 1000 * 100,"%")
print("自然物画像",sum_s / 1000 * 100,"%")
print("全体画像",(sum_s + sum_j) / 2000 * 100,"%")


人工物画像 94.8 %
自然物画像 90.4 %
全体画像 92.60000000000001 %


In [8]:
# 正誤表
print('Cross tabulation')
y_pred = model.predict_classes(x_test)
pd.crosstab(y_pred, Y_eva) 

Cross tabulation


Exception: Data must be 1-dimensional

In [ ]:
print(x_test.shape)
X_test = x_test.reshape(x_test.shape[0], 3, 64, 64)
print(X_test.shape)

X = np.asarray(X)
print(X.shape)
X = X.transpose(0, 3, 2, 1)
X = X.astype(np.int32)
print(X.shape)

# そこで実際に、間違いの数字を見る。
# 青ラベル：実際の数字；赤ラベル：間違えて予測した数字
test_wrong = [im for im in zip(X_test, y_pred, Y_eva)
              if im[1] != im[2]]
print(len(test_wrong))

plt.figure(figsize=(64, 64))
for ind, val in enumerate(test_wrong):
    plt.subplots_adjust(left=0, right=1, top=1, bottom=0)
    plt.subplot(20, 20, ind + 1)
    im = 1 - val[0][0]
    plt.axis("off")
    plt.imshow(im)
    plt.text(8, 0, val[1], fontsize=50, color='red')
    plt.text(0, 0, val[2], fontsize=50, color='blue')
#     break